In [1]:
#!pip install pandas SQLAlchemy
#!pip install pymysql

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import text, String
from sqlalchemy.types import Integer

# Configuration de la connexion MySQL
def get_engine(user='root', password='Marley08092022!', host='127.0.0.1', database='platforms'):
    return create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Générer un identifiant unique dans le format 'tt0109830' et vérifier l'unicité
def generate_unique_ids(df, existing_ids):
    unique_ids = []
    base_number = 1000000  # Pour générer les ID dans le format souhaité

    # Générer des identifiants uniques en évitant les doublons
    for i in range(len(df)):
        new_id = f'tt{str(base_number + i)[1:]}'
        while new_id in existing_ids or new_id in unique_ids:
            base_number += 1
            new_id = f'tt{str(base_number + i)[1:]}'
        unique_ids.append(new_id)
    
    return unique_ids

# Fonction pour charger et préparer les données
def prepare_data(file_path, platform_name, columns_to_drop=['imdbNumVotes']):
    # Charger le fichier CSV
    df = pd.read_csv(file_path)

    df = df.dropna(subset=['title'])
    
    # Ajouter la colonne 'platform' en fonction du nom de la plateforme
    df['platform'] = platform_name
    
    # Supprimer les colonnes non nécessaires
    df = df.drop(columns=columns_to_drop, errors='ignore')
    
    # Conversion de 'releaseYear' en type numérique
    df['releaseYear'] = pd.to_numeric(df['releaseYear'], errors='coerce').astype('Int64')
    
    # Filtrer uniquement les films
    df = df[df['type'] == 'movie'].drop(['type'], axis=1, errors='ignore')
    
    # Renommer les colonnes pour correspondre au schéma cible
    df = df.rename(columns={
        'imdbAverageRating': 'rating',
        'availableCountries': 'countries',
        'imdbId': 'platform_id'
    })
    
    # Identifier les IDs existants et remplacer les NaN dans 'platform_id'
    existing_ids = set(df['platform_id'].dropna())
    unique_ids = generate_unique_ids(df[df['platform_id'].isna()], existing_ids)
    
    # Remplacer les NaN avec les identifiants uniques générés
    df.loc[df['platform_id'].isna(), 'platform_id'] = pd.Series(unique_ids, index=df[df['platform_id'].isna()].index)
    
    # S'assurer que platform_id est de type VARCHAR en définissant explicitement sa longueur
    df['platform_id'] = df['platform_id'].astype(str)
    
    return df

# Fonction pour sauvegarder le DataFrame dans MySQL
def save_to_sql(df, table_name, engine):
    # Forcer le type 'platform_id' à VARCHAR(20)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False, dtype={'platform_id': String(20), 'releaseYear': Integer()})
    
    # Ajouter la clé primaire
    with engine.connect() as connection:
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY (platform_id);"))


# Fonction principale pour exécuter tout le processus
def process_and_save(file_path):
    # Extraire le nom de la plateforme à partir du nom du fichier
    platform_name = file_path.split('/')[-1].split('.')[0]
    
    # Préparer les données
    df = prepare_data(file_path, platform_name)
    
    # Sauvegarder les données dans la table MySQL
    engine = get_engine()
    save_to_sql(df, platform_name, engine)


In [3]:
file_paths = [
    'plateforms/netflix.csv',
    'plateforms/prime.csv',
    'plateforms/hulu.csv',
    'plateforms/hbo.csv',
    'plateforms/apple.csv'
]

for file_path in file_paths:
    process_and_save(file_path)